In [41]:
import numpy as np
import random
from collections import Counter
from datetime import datetime
import keras
import sys
import io
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from datetime import datetime
%reload_ext tensorboard

In [4]:
with open('data/Mushrooms.txt', 'r') as mushies:
    mushroom_accounts = mushies.read()

with open('data/Cannabis.txt', 'r') as pot:
    cannabis_accounts = pot.read()
    
with open('data/MDMA.txt', 'r') as molly:
    mdma_accounts = molly.read()
    
with open('data/LSD.txt', 'r', encoding='utf-8') as acid:
    lsd_accounts = acid.read()

In [66]:
def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)                     #


#returns a model trained on whatever text it is given. 
def make_model(text):
    chars = sorted(list(set(text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(text) - seqlen - 1, step):
        sentences.append(text[i: i + seqlen + 1])

    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)

    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1


    logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

    model = Sequential()
    model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dense(len(chars), activation='softmax'))


    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.1),
        metrics=['categorical_crossentropy', 'accuracy']
    )


    def on_epoch_end(epoch, _):
        """Function invoked at end of each epoch. Prints generated text."""
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - seqlen - 1)

        diversity = 0.5
        
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model.fit(x, y,
        batch_size=128,
        epochs=15,
        callbacks=[print_callback])

    return model, x, y

    #%tensorboard --logdir logs/scalars

In [67]:
mushrooms, mush_x, mush_y = make_model(mushroom_accounts[:len(mushroom_accounts)//2])

Epoch 1/15
62585/62585 [==============================] - 113s 2ms/step - loss: 8.0134 - categorical_crossentropy: 8.0134 - accuracy: 0.1886

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "an a brief moment I felt wet and dirty. "
an a brief moment I felt wet and dirty.  I from look on crop to lo a to look on to look to on to look to to to look to look to lo to trip to look to to look the in look or look look or or tirl ot to on look to look look to look to for the look to look to to to look leat to plott to look to for on to to look to lo the look lig the look to to the frient to to look we re look the or look to to to to plont to to the to on look take to do lo
Epoch 2/15
62585/62585 [==============================] - 90s 1ms/step - loss: 1.7279 - categorical_crossentropy: 1.7279 - accuracy: 0.4973

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "e two more 1.25g doses instead of just o"
e two more 1.25g d

C:\Users\e_san_hjm\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


trange the can the watch get the beautiful the heart the story me and the hand the expected on the hard and the move the hands that was the way have seemed to come same the ground at the compected many the manter, the halls
Epoch 5/15
62585/62585 [==============================] - 85s 1ms/step - loss: 1.5399 - categorical_crossentropy: 1.5399 - accuracy: 0.5452

----- Generating text after Epoch: 4
----- diversity: 0.5
----- Generating with seed: "y ego. The present reality would fade, a"
y ego. The present reality would fade, and we mebic and and more me we were blue she was suddenly we were such eat the feeling of my buds of the whole the mushroom and we walked was the state the time the same was seemed he was when me me. 

After one with the stomach other was very sitting and which we were me the much of the time chosed stomach we were so we should dose completely my experience we were very such on my wonder world and
Epoch 6/15
62585/62585 [==============================] - 85s 1ms

In [35]:
mdma, mdma_x, mdma_y = make_model(mdma_accounts)

Epoch 1/2
41238/41238 [==============================] - 56s 1ms/step - loss: 4.2157 - categorical_crossentropy: 4.2157 - accuracy: 0.1512: 4s - loss: 4.3629 - categorical_crossentro

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "dancing again, and P's bowl shattered on"
dancing again, and P's bowl shattered on t nd wave we wind wa pt ind pre ind wat tnd tin we the tind win  t  w t an wan war wat wary tick rand the wo  d wict an wind th un to to t expering wing we  wand win wr ind go ts und wi w and wo we f an t nc  wat for ar wand wo p t+ and the ind the fe wang  I wing bon t five wo w and wo thor ind and s wing con d waro ware wang wo  cing wand wor ind d me wand walind nnd and wat wore winc win we ta
Epoch 2/2
41238/41238 [==============================] - 55s 1ms/step - loss: 1.7526 - categorical_crossentropy: 1.7526 - accuracy: 0.4935

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: " as LaTeX (for geek

In [70]:
# evaluate trained model, to check saved and retreived model later
def eval_model(model, x, y):
    print(model.evaluate(x[-2000:], y[-2000:], verbose=0))

In [71]:
def save_trained_model(trained_model, drug_name):
    now = datetime.now()
    model_json = trained_model.to_json()
    filename = drug_name + "_" + now.strftime("%m-%d_%H-%M-%S")
    with open("saved_models/" + filename + ".json", "w+") as json_file:
        json_file.write(model_json)
    # serialize weights
    trained_model.save_weights("saved_models/" + filename + ".h5")
    print(drug_name + "_" + now.strftime("%m-%d_%H-%M-%S") + " model saved!")

In [72]:
def load_trained_model(filename):
    # load model
    json_file = open("saved_models/" + filename + ".json", "r")
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json)
    # load weights
    loaded_model.load_weights("saved_models/" + filename + ".h5")
    print(filename + " model loaded")
    # compile
    loaded_model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.1),
        metrics=['categorical_crossentropy', 'accuracy']
    )
    return loaded_model

In [73]:
eval_model(mushrooms, mush_x, mush_y)
save_trained_model(mushrooms, "mushrooms")

[1.505263566017151, 1.5052634477615356, 0.557812511920929]
mushrooms_11-30_16-56-31 model saved!


In [75]:
# hardcode filename here
model_loaded = load_trained_model("mushrooms_11-30_16-56-31")
eval_model(model_loaded, mush_x, mush_y)

mushrooms_11-30_16-56-31 model loaded
[1.505263566017151, 1.5052634477615356, 0.557812511920929]
